In [100]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import knn

In [101]:
df = pd.read_csv('data/heart_2020_small.csv')
df

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,28.15,No,No,No,7,0,Yes,Female,80 or older,White,No,No,Good,7,No,No,No
1,Yes,25.06,No,No,No,0,0,Yes,Female,80 or older,White,Yes,No,Good,7,No,No,Yes
2,No,23.67,Yes,No,Yes,0,0,No,Male,75-79,White,No,Yes,Very good,9,No,No,Yes
3,No,24.72,Yes,No,No,0,0,No,Female,65-69,Black,No,Yes,Very good,7,No,No,No
4,No,21.03,No,No,No,0,0,No,Female,80 or older,White,No,No,Very good,8,No,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,No,32.92,Yes,No,No,0,0,Yes,Female,55-59,Hispanic,Yes,No,Fair,6,No,No,No
3193,No,25.61,No,No,No,0,0,No,Female,25-29,Hispanic,No,Yes,Excellent,8,No,No,No
3194,No,33.72,No,No,No,0,0,No,Male,60-64,Hispanic,No,Yes,Very good,9,No,No,No
3195,No,29.76,No,No,No,0,0,No,Female,45-49,Hispanic,No,No,Very good,4,No,No,No


In [102]:
# Se sustituyen valores categóricos por valores numéricos

dfNum = pd.DataFrame()
for col, t in zip(df.columns, df.dtypes):
    pos = 0
    keys = pd.DataFrame()    
    if (t == 'object'):
        key = dict([(k,i) for i,k in enumerate(df.loc[:, col].unique())])
        dfNum[col] = df[col].map(key)
    else:
        dfNum[col] = df[col]

In [103]:
# Se filtran las columnas redundantes
r = 0.0
corrMat = dfNum.corr()
relevantCols = corrMat.loc[(corrMat['HeartDisease'] > r) | (corrMat['HeartDisease'] < -r)].transpose().columns
dfNum = dfNum.loc[:, relevantCols.to_list()]

In [104]:
# Se separan la columna a predecir de los atributos

individuals = pd.DataFrame(dfNum.iloc[:,1:])
individuals

,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,28.15,0,0,0,7,0,0,0,0,0,0,0,0,7,0,0,0
1,25.06,0,0,0,0,0,0,0,0,0,1,0,0,7,0,0,1
2,23.67,1,0,1,0,0,1,1,1,0,0,1,1,9,0,0,1
3,24.72,1,0,0,0,0,1,0,2,1,0,1,1,7,0,0,0
4,21.03,0,0,0,0,0,1,0,0,0,0,0,1,8,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,32.92,1,0,0,0,0,0,0,10,2,1,0,2,6,0,0,0
3193,25.61,0,0,0,0,0,1,0,7,2,0,1,3,8,0,0,0
3194,33.72,0,0,0,0,0,1,1,8,2,0,1,1,9,0,0,0
3195,29.76,0,0,0,0,0,1,0,4,2,0,0,1,4,0,0,0


In [105]:
target = pd.DataFrame(dfNum['HeartDisease'])
target

,HeartDisease
0,0
1,1
2,0
3,0
4,0
...,...
3192,0
3193,0
3194,0
3195,0


In [95]:
# Normalización
individuals = pd.DataFrame((individuals - individuals.min())/(individuals.max() - individuals.min()))
individuals


,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease
0,0.0,0.211857,0.0,0.0,0.0,0.233333,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.400000,0.0,0.0
1,1.0,0.166174,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.333333,0.0,0.00,0.400000,0.0,0.0
2,0.0,0.145624,1.0,0.0,1.0,0.000000,0.0,1.0,1.0,0.083333,0.0,0.000000,1.0,0.25,0.533333,0.0,0.0
3,0.0,0.161147,1.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.166667,0.2,0.000000,1.0,0.25,0.400000,0.0,0.0
4,0.0,0.106594,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.000000,0.0,0.000000,0.0,0.25,0.466667,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,0.0,0.282377,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.833333,0.4,0.333333,0.0,0.50,0.333333,0.0,0.0
3193,0.0,0.174305,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.583333,0.4,0.000000,1.0,0.75,0.466667,0.0,0.0
3194,0.0,0.294205,0.0,0.0,0.0,0.000000,0.0,1.0,1.0,0.666667,0.4,0.000000,1.0,0.25,0.533333,0.0,0.0
3195,0.0,0.235659,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.333333,0.4,0.000000,0.0,0.25,0.200000,0.0,0.0


In [106]:
# Se separa el conjunto de datos en entrenamiento y test

xTrain, xTest, yTrain, yTest = train_test_split(individuals, target, test_size=0.3)
classes = target['HeartDisease'].unique()
classes


array([0, 1])

In [107]:
# Se instancia el clasificador y se añaden los datos

classifier = knn.Knn(200)
classifier.fit(xTrain, yTrain)
# classifier.predSingle(yTrain[0])

In [108]:
# classifier.predSingle(xTrain.iloc[1])

In [109]:
predicted = classifier.pred(xTest)
precision, confusionMatrix = classifier.precision(predicted, yTest, classes)
confusionMatrix = pd.DataFrame(confusionMatrix, columns=np.concatenate((np.array(["Class"]), classes)))
print("Fila = predicho, Columna = real")
print("Precisión = " + str(precision))
confusionMatrix


Fila = predicho, Columna = real
Precisión = 0.91875


,Class,0,1
0,0.0,882.0,78.0
1,1.0,0.0,0.0
